In [43]:
from standardInclude import *

In [44]:
bvs = ["plusPoint2",
      "minusPoint2",
       "plusPoint1",
       "minusPoint1",
       "original",
       "originalEnd"
      ]

for biasVoltageString in bvs:
    print("Working on", biasVoltageString)
    fin = "./filterWheel_GainCalibration/biasVoltageGains_"+biasVoltageString+".txt"


    # In[106]:


    calo = 0
    xtal = -1
    gainConstants = []
    with open(fin) as input_file:
        for i, line in enumerate(input_file):
            #print(line)
            if("calo" in line):
                calo = int(line.split('calo')[1].split(' ')[0])
            if("xtal" in line):
                xtal = int(line.split(" ")[4].split("xtal")[1])
                constant = float(line.split(" ")[6].split("\n")[0])
                gainConstants.append([calo, xtal, constant])


    # ### Read in the current gain settings

    # In[107]:


    iteration = 6
    filePath = "./outputGainSettings/iteration"+str(iteration)+"/"
    #filePath = "./outputGainSettings/biasVoltageScans/original/"
    settingsByCalo = []

    for calo in range(1,25):
        settingsi = [-1 for i in range(54)]
        fin2 = filePath+"gainSettings_Iteration"+str(iteration)+"_calo"+str(calo)+".json"
        #fin2 = filePath+"gainSettings_original_calo"+str(calo)+".json"
        with open(fin2) as input_file:
            for i, line in enumerate(input_file):
                #print(line)
                if("calo" in line):
                    #print(line)
                    calo = int(line.split("calo")[1].split('"')[0])
                if("sipm" in line):
                    #print(line)
                    sipm = int(line.split('"')[1].split("sipm")[1])
                    #print(sipm)
                    #print(line.split(" "))
                    setting = float(line.split(" ")[1].split(",")[0])
                    settingsi[sipm] = setting
                    for constants in gainConstants:
                        if((constants[0] == calo) and (constants[1] == sipm)):
                            constants.append(setting)


    # In[108]:


    print(gainConstants[:10],"...")


    # In[109]:


    def si( ri ):
        #return 1.3 - 80*r.TMath.Log10( ri )
        return 104 - 80*r.TMath.Log10( ri )
        #return 80*math.log(10*10**(0.3)/ri)/2.30259   

    def ri( sold ):
        return 10**( (1/20.)*(26 - sold/4.) )

    def calculateSetting( constant, currentSetting ):
        goalSetting = 0.28

        #rs = 10**((1/20.)*(26 - currentSetting/4.))
        rs = ri(currentSetting)

        #s = 1.3 - 80*r.TMath.Log10(ri)
        #s = currentSetting
        #snew = s*(constant/goalSetting * ri)
        snew = si( constant/goalSetting * rs )

        if(False):
            print("Current setting:", currentSetting)
            print("Constant:", constant)
            print("r(s) = ", rs)
            print("New setting", snew)
            print(" ")

        return snew


    # In[110]:


    calcConstants = []
    for constants in gainConstants:
        calcConstants.append( calculateSetting(constants[2], constants[3]) )


    # In[112]:


    calos, xtals, initialConstants, initialSettings = zip(*gainConstants)


    # In[118]:


    settingsToApply = []
    for gain in calcConstants:
        settingsToApply.append( gain )


    # In[119]:


    #biasVoltageString = "original"
    filePath = "./outputGainSettings/biasVoltageScans/"+biasVoltageString+"/"
    for i, gainSetting in enumerate(settingsToApply):
    #for i, gainSetting in enumerate(calcConstants):
        calo = int(np.floor((i) / 54) + 1)
        fileName="gainSettings_"+biasVoltageString+"_calo"+str(calo)+".json"
        #print(fileName)
        sipm = i % 54
        if(sipm == 0):
            outfile = open(filePath+fileName,"w") 
            #print("{")
            outfile.write("{"+"\n")
        #print("   \"sipm"+str(sipm)+"\": "+str(gainSetting))
        #outfile.write("   \"sipm"+str(sipm)+"\": "+str(gainSetting)+",\n")
        endchar=",\n"
        if(sipm == 53):
               endchar="\n" 
        outfile.write("\t\"sipm"+str(sipm)+"\": "+str(gainSetting)+endchar)
        if(sipm == 53):
            #print("}")
            outfile.write("}")
            outfile.close()


Working on plusPoint2
[[1, 0, 0.286, 63.0], [1, 1, 0.226, 71.0], [1, 2, 0.231, 56.0], [1, 3, 0.25, 67.0], [1, 4, 0.25, 65.0], [1, 5, 0.222, 60.0], [1, 6, 0.245, 63.0], [1, 7, 0.247, 43.0], [1, 8, 0.279, 66.0], [1, 9, 0.179, 80.0]] ...
Working on minusPoint2
[[1, 0, 0.329, 63.0], [1, 1, 0.314, 71.0], [1, 2, 0.284, 56.0], [1, 3, 0.291, 67.0], [1, 4, 0.304, 65.0], [1, 5, 0.294, 60.0], [1, 6, 0.308, 63.0], [1, 7, 0.307, 43.0], [1, 8, 0.321, 66.0], [1, 9, 0.277, 80.0]] ...
Working on plusPoint1
[[1, 0, 0.275, 63.0], [1, 1, 0.26, 71.0], [1, 2, 0.242, 56.0], [1, 3, 0.256, 67.0], [1, 4, 0.232, 65.0], [1, 5, 0.236, 60.0], [1, 6, 0.239, 63.0], [1, 7, 0.222, 43.0], [1, 8, 0.284, 66.0], [1, 9, 0.206, 80.0]] ...
Working on minusPoint1
[[1, 0, 0.357, 63.0], [1, 1, 0.278, 71.0], [1, 2, 0.31, 56.0], [1, 3, 0.283, 67.0], [1, 4, 0.297, 65.0], [1, 5, 0.279, 60.0], [1, 6, 0.306, 63.0], [1, 7, 0.281, 43.0], [1, 8, 0.348, 66.0], [1, 9, 0.278, 80.0]] ...
Working on original
[[1, 0, 0.271, 63.0], [1, 1, 0.289